In [23]:
# 📌 Importar librerías
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 📌 Rutas de entrada y salida
clean_path = "C:/Users/pmate/OneDrive - Universidad San Francisco de Quito/8 SEMESTRE/DataMining/Examanes/Proyecto_Megaline/data/clean/"
ml_path = "C:/Users/pmate/OneDrive - Universidad San Francisco de Quito/8 SEMESTRE/DataMining/Examanes/Proyecto_Megaline/data/ml/"

# 📌 Cargar el dataset con consumo y costos
df_users = pd.read_csv(clean_path + "fact_consumo.csv")

# ✅ **1️⃣ Creación de Variables Relativas**
df_users["uso_minutos_relativo"] = df_users["total_minutos"] / df_users["minutes_included"]
df_users["uso_sms_relativo"] = df_users["total_sms"] / df_users["messages_included"]
df_users["uso_datos_relativo"] = df_users["total_mb"] / df_users["mb_per_month_included"]

# Reemplazamos valores infinitos por 1 (usuarios sin consumo no generan división por cero)
df_users.replace([np.inf, -np.inf], 1, inplace=True)

# ✅ **2️⃣ Excesos Normalizados**
df_users["porcentaje_exceso_minutos"] = df_users["exceso_minutos"] / df_users["minutes_included"]
df_users["porcentaje_exceso_sms"] = df_users["exceso_sms"] / df_users["messages_included"]
df_users["porcentaje_exceso_mb"] = df_users["exceso_mb"] / df_users["mb_per_month_included"]

# Reemplazamos NaN por 0 (usuarios que no excedieron no tienen valores en estas columnas)
df_users.fillna(0, inplace=True)

# ✅ **3️⃣ Transformaciones de Variables**
df_users["log_total_mb"] = np.log1p(df_users["total_mb"])  # log1p evita log(0)
df_users["log_costo_total"] = np.log1p(df_users["costo_total"])

# Escalado de 0 a 1 para algunas variables
scaler = MinMaxScaler()
cols_to_scale = ["total_minutos", "total_sms", "total_mb", "costo_total"]
df_users[cols_to_scale] = scaler.fit_transform(df_users[cols_to_scale])

# ✅ **4️⃣ Variables Demográficas**
df_users["segmento_edad"] = pd.cut(df_users["age"], bins=[0, 29, 60, 100], labels=["Joven", "Adulto", "Mayor"])

# Agrupación de regiones en categorías más generales (ejemplo simplificado)
df_users["region_grupo"] = df_users["city"].apply(lambda x: "Este" if "New York" in x else ("Oeste" if "Los Angeles" in x else "Centro"))

# ✅ **5️⃣ Guardamos el Dataset Final en `data/ml/`**
df_users.to_csv(ml_path + "usuarios_feature_engineering.csv", index=False)

print("\n✅ Feature Engineering completado. Archivo guardado en 'data/ml/'.")



✅ Feature Engineering completado. Archivo guardado en 'data/ml/'.


In [24]:
# 📌 Importar librerías
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 📌 Ruta del dataset después de Feature Engineering
ml_path = "C:/Users/pmate/OneDrive - Universidad San Francisco de Quito/8 SEMESTRE/DataMining/Examanes/Proyecto_Megaline/data/ml/"
df = pd.read_csv(ml_path + "usuarios_feature_engineering.csv")

# ✅ **1️⃣ Eliminar Variables Redundantes**
df.drop(columns=[
    "exceso_minutos", "exceso_sms", "exceso_mb",  # Reemplazadas por variables relativas
    "costo_extra_minutos", "costo_extra_sms", "costo_extra_mb",  # Redundantes con el costo total
    "minutes_included", "messages_included", "mb_per_month_included",  # Redundantes con usd_monthly_pay
    "usd_per_minute", "usd_per_message", "usd_per_gb",  # Altamente correlacionadas
    "log_costo_total", "log_total_mb"  # Si costo_total ya está escalado, no es necesario
], inplace=True)

# ✅ **2️⃣ Normalización de Variables**
scaler = MinMaxScaler()
cols_to_scale = ["total_minutos", "total_sms", "total_mb", "costo_total"]
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

# ✅ **3️⃣ Guardar el dataset corregido**
df.to_csv(ml_path + "usuarios_feature_engineering_corregido.csv", index=False)

print("\n✅ Corrección de colinealidad completada. Archivo guardado en 'data/ml/'.")



✅ Corrección de colinealidad completada. Archivo guardado en 'data/ml/'.


In [ ]:
#ELIMINAMOS FEATURES CON ALTA CORRELACION PARA EVITAR ERRORES AL ENTRENAR 